<a href="https://colab.research.google.com/github/yukinaga/automl/blob/main/section_3/02_nlp_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AutoMLによる「自然言語処理」 -分類-
引き続き、AutoMLによる「自然言語処理」を行います。  
今回は、クラウドファンディングによる融資が、返済可能であるか判定するモデルを作成します。

## PyCaretのインストール
AutoMLをサポートするライブラリ、PyCaretをバージョンを指定してインストールします。

In [ ]:
!pip install pycaret==2.3.1

## Google Colabの設定
Google Colab環境でPyCaretのインタラクティブな要素を表示するためには、以下のコードを実行する必要があります。

In [ ]:
from pycaret.utils import enable_colab

enable_colab()

## データセットの読み込み
今回も、「Kiva Microfunds」のデータを使用します。  

* country: 借り手の国
* en: 借り手の個人的なストーリー
* gender: 性別 (M=男性, F=女性)
* loan_amount: 承認された融資の金額
* nonpayment: 貸し手のタイプ（Lender＝Kivaに登録した個人ユーザー、Partner＝Kivaと協力するマイクロファイナンス機関)
* sector: 借り手の業種
* status: 融資のステータス(1-債務不履行, 0-返済)
 
https://www.kiva.org/ 

In [ ]:
from pycaret.datasets import get_data

data = get_data("kiva")  # データの取得
data.shape  # データの形状

このままだとハイパーパラメータの最適化や結果の可視化に時間がかかるので、サンプルを1000抜き出して使用します。  
時間がかかっても問題ないのであれば、以下のコードはコメントアウトしてください。

In [ ]:
data = data.sample(1000, random_state=786).reset_index(drop=True)
data.shape

## 環境の設定
PyCaretの環境を設定します。  
今回は、custom_stopwordsによりストップワードを設定します。 
ストップワードあまりに一般的であるため、トピックの特徴付けに寄与しない単語です。  
以下の例では、「01_nlp_copus.ipynb」で得られた、高い頻度で得られた単語をストップワードとしています。   
これらの単語は、文章内で非常に高い頻度で使用されおり、情報よりもノイズを増やしています。  

In [ ]:
from pycaret.nlp import setup

stopwords = ["loan", "income", "usd", "many", "also", "make", "business", "buy", "sell",
             "purchase", "year", "people", "able", "enable", "old", "woman", "child", "school"]

clf = setup(data=data, target="en", 
            session_id=123,
            custom_stopwords=stopwords)  # 環境の初期化

## モデルの作成

トピックモデルの1つLatent Dirichlet Allocation (LDA)のモデルを作成し、訓練します。  

In [ ]:
from pycaret.nlp import create_model

lda = create_model("lda", num_topics=5, multi_core = True)  # LDAのモデルを作成し訓練

plotに"topic_distribution"を指定し、各トッピックに分類されたサンプル数を確認します。  

In [ ]:
from pycaret.nlp import plot_model

plot_model(lda, plot="topic_distribution")

## ハイパーパラメータの最適化
tune_model()関数を使って、トピック数を最適化します。  
以下のコードでは、トピックモデルの評価指標としてよく使われるCoherenceが最も高くなるように、トピック数が最適化されます。  
Coherenceは、トピック内に含まれる語に一貫性があり、人間がいかに理解しやすいトピックであるかを表します。  
Coherenceの具体的な計算方法については、以下などを参考にしてください。  
https://svn.aksw.org/papers/2015/WSDM_Topic_Evaluation/public.pdf

また、custom_gridにはトピック数の候補を設定することができます。  

In [ ]:
from pycaret.nlp import tune_model

tuned_lda = tune_model(model = "lda", multi_core = True, custom_grid=[2, 3, 4, 5, 6, 7, 8])

plotに"topic_distribution"を指定すると、各トッピックに分類されたサンプル数をグラフで確認することができます。  
各トピックの棒にカーソルを当てて、キーワードが適切になっていることを確認しましょう。

In [ ]:
plot_model(tuned_lda, plot="topic_distribution")

t-SNE（T-Distributed Stochastic Neighbor Embedding ）により、データを3次元で可視化しましょう。  

In [ ]:
plot_model(tuned_lda, plot="tsne")

## モデルの保存と読み込み

`save_model`関数によりモデルを保存することができます。  

In [ ]:
from pycaret.nlp import save_model

save_model(tuned_lda, "lda_model")

`load_model`関数により、保存されたモデルを読み込むことができます。

In [ ]:
from pycaret.nlp import load_model

loaded_lda = load_model("lda_model")

## （補足）ラベルを使った分類
今回使用しているデータセットは、"status"の列によりラベル付けされています。  
statusが1は債務不履行で、0は返済済みです。  
tune_model()関数は、このようなラベルを正解として使って機械学習を行い、最適なトピック数を決定することができます。  
分類問題なので、最適化に使用する指標はAccuracy（正解率）です。  
  
以下のコードでは、supervised_targetに"status"を指定することで正解ラベルの列を指定しています。  
また、estimatorに"lr"を指定しており、ロジスティック回帰による機械学習を選択していることになります。  

tune_model関数の詳しい使い方については、以下の公式ドキュメントを参考にしてください。  
https://pycaret.org/tune-model/

In [ ]:
tuned_classification = tune_model(model = "lda",
                                  multi_core = True,
                                  supervised_target = "status",
                                  estimator = "lr",
                                  custom_grid=[2, 3, 4, 5, 6, 7, 8])

plotに"topic_distribution"を指定し、各トッピックに分類されたサンプル数をグラフで確認します。

In [ ]:
plot_model(tuned_classification, plot="topic_distribution")

t-SNEにより、データを3次元で可視化します。

In [ ]:
plot_model(tuned_classification, plot="tsne")